In [1]:
%load_ext autoreload
%autoreload 2


%matplotlib inline
%cd C:\MAD4AG

from tqdm.auto import tqdm
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')

C:\MAD4AG


In [2]:
filename = r'.\dbs\twins\multiple_days_03-15.pkl'

combined_df= pd.read_pickle(filename)
#combined_df.columns


In [3]:
df_clusters=pd.read_parquet(f'./dbs/intermediate/df_selected_clusters_activities.parquet')

In [4]:
df_survey_act = pd.read_pickle(f'./dbs/intermediate/survey_act_weekday_modified.pkl')

## generate activity schedule for simulation #1
- by merging survey activity sequence table
- the survey activity sequence table brings activity sequences and activity start end times

In [5]:
day = '0'

In [6]:
df_act_schedule_0 = pd.merge(combined_df[['uid', 'employee', 'Deso', 'wt', 'wt_p', 'sub_id_'+day,'act_seq_simp_'+day]], df_survey_act[['sub_id', 'purpose','h_s','h_e']], left_on='sub_id_'+day,right_on='sub_id')

df_act_schedule_0['act_seq'] = df_act_schedule_0.groupby('uid').cumcount()

# replace school activities with work activity
df_act_schedule_0['purpose']= df_act_schedule_0['purpose'].replace({'School': 'Work'})

### add sequence information of other activities

In [7]:
df_act_schedule_0['other_order']= 0

df_act_schedule_0['other_order'][df_act_schedule_0.purpose=='Other'] = df_act_schedule_0[df_act_schedule_0.purpose=='Other'].groupby('uid').cumcount()

df_act_schedule_0['other_order']=df_act_schedule_0['other_order'].astype(str)

df_act_schedule_0['purpose'][df_act_schedule_0.purpose=='Other']= df_act_schedule_0['purpose'][df_act_schedule_0.purpose=='Other'] + df_act_schedule_0['other_order'][df_act_schedule_0.purpose=='Other']

df_act_schedule_0.drop(columns=['other_order'] ,inplace=True)


## re-organise df_cluster table by the other activities order and repetition

In [8]:
df_clusters_0 = df_clusters[['uid','cluster','other_clusters_'+day,'selection_'+day, 'X','Y']][df_clusters.selection_0 != 'non']

In [9]:
df_clusters_0_other = df_clusters_0[df_clusters_0['selection_'+day] =='Other']


df_other_org = df_clusters_0_other[['uid', 'other_clusters_'+day]]
df_other_org= df_other_org.drop_duplicates(subset='uid')


In [10]:
# it will transform each element of the list in the cloumn 'other_cluster_O' to a row.

df_other_org = df_other_org.explode('other_clusters_'+day)

df_other_org.rename(columns={'other_clusters_'+day:'cluster'}, inplace=True)

df_other_org['other_order'] = df_other_org.groupby('uid').cumcount()

In [11]:
df_other_org = pd.merge(df_other_org, df_clusters_0_other, on=['uid', 'cluster'],how='left')

# add sequence information of other activities

df_other_org['other_order'] = df_other_org['other_order'].astype(str)

df_other_org['selection_'+day]= df_other_org['selection_'+day]+ df_other_org['other_order']

df_other_org.drop(columns=['other_order'] ,inplace=True)

In [12]:
df_clusters_0 = df_clusters_0[df_clusters_0['selection_'+day] !='Other']

In [13]:
df_clusters_0 = pd.concat([df_clusters_0, df_other_org], ignore_index=True)

In [14]:
df_clusters_0.drop(columns=['other_clusters_'+day] ,inplace=True)

In [15]:
df_act_schedule_0 = pd.merge(df_act_schedule_0, df_clusters_0, left_on=['uid','purpose'],right_on=['uid','selection_'+day], how='left')

In [16]:
df_act_schedule_0.drop(columns=['selection_'+day] ,inplace=True)


In [17]:
file = r'.\dbs\twins\%s-schedule-%s.pkl'%(filename[26:31],day)

df_act_schedule_0.to_pickle(file)

In [ ]:
# df_act_schedule = pd.DataFrame(combined_df.act_seq_simp_0.tolist(), index=combined_df['uid'])
#
# df_act_schedule = df_act_schedule.stack()
#
# df_act_schedule = pd.DataFrame(df_act_schedule)
#
# df_act_schedule = df_act_schedule.reset_index()
#
# df_act_schedule = df_act_schedule.rename(columns={'level_0': 'PId', 'level_1': 'act_id', 0: 'act_purpose'})